# Baseline (01)

# Imports

In [1]:
!pip install "git+http://github.com/snowdd1/wcs.git" >/dev/null

  Running command git clone -q http://github.com/snowdd1/wcs.git /tmp/pip-req-build-vt_porwt


In [2]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from wcs.google import google_drive_share
import pandas as pd
import urllib.request
from urllib.parse import urlparse
import os
import warnings
warnings.simplefilter(action='ignore')

In [3]:

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#config = ConfigProto()
#tf.config.gpu_options.allow_growth = True
#session = InteractiveSession(config=config)

# Load Data

In [5]:
data_dir_url = 'https://drive.google.com/file/d/1etygmCq28-9yWBhlFwXvFqPkcBKaq8Ow/view?usp=sharing'  # zip

DATA_DIR = '../data/external/images'

In [6]:
if not os.path.exists(DATA_DIR):
  with open('data_dir.zip', 'wb') as f:
    f.write( google_drive_share(data_dir_url, binaryfile=True).read() )

  !unzip data_dir.zip > /dev/null

# Preproc

In [7]:
def retrieveFilename(url):
    a = urlparse(url)
    picfilename = os.path.basename(a.path)
    return picfilename


In [8]:
df = pd.read_parquet("../data/raw/df.parquet.gzip")
df["filename"] = df.loc[~df["poster_url"].isnull(),"poster_url"].apply(retrieveFilename)
df.head()

,adult,id,original_title,popularity,video,url,poster_url,genre_id,filename
380000,False,537245,El Pescador,0.6,False,https://www.themoviedb.org/movie/537245,https://www.themoviedb.org/t/p/w500//pOOabahFF...,[],pOOabahFFsee0ojSr3nBnUIfgD0.jpg
380001,False,537246,Der Herzog von Aleria,0.6,False,https://www.themoviedb.org/movie/537246,None,[18],NaN
380002,False,537247,Wem nie durch Liebe Leid geschah!,0.6,False,https://www.themoviedb.org/movie/537247,None,[18],NaN
380003,False,537250,¿Por qué ya no me quieres?,1.4,False,https://www.themoviedb.org/movie/537250,https://www.themoviedb.org/t/p/w500//8Y0aOxqSe...,"[80, 18]",8Y0aOxqSetK3hXKPFDTrHwu6dWa.jpg
380004,False,537251,民暴の帝王,0.6,False,https://www.themoviedb.org/movie/537251,https://www.themoviedb.org/t/p/w500//5ayhYaAfg...,[],5ayhYaAfgXfthbdZYn2MnkxGOew.jpg


In [9]:
#df = pd.read_csv('data/movies.csv')

In [10]:
# use only mot null rows
df = df.dropna()
# set data path to basename of the file
#df['data_path'] = df['poster_path'].apply(lambda x: x.split('/')[-1])
# remove rows with empty genre_id list and set correct list type
df['genre_id'] = df['genre_id'].apply(lambda x: np.nan if len(eval(x)) == 0 else x)
df = df.dropna()
df['genre_id'] = df['genre_id'].apply(lambda x: eval(x))

print(f'len of df: {len(df)}')

len of df: 267974


In [11]:
df.head()

,adult,id,original_title,popularity,video,url,poster_url,genre_id,filename
380003,False,537250,¿Por qué ya no me quieres?,1.40,False,https://www.themoviedb.org/movie/537250,https://www.themoviedb.org/t/p/w500//8Y0aOxqSe...,"[80, 18]",8Y0aOxqSetK3hXKPFDTrHwu6dWa.jpg
380010,False,537266,Success,0.60,False,https://www.themoviedb.org/movie/537266,https://www.themoviedb.org/t/p/w500//tJMm1JJUe...,[18],tJMm1JJUeWOLb4nPDBD6sZvFYU3.jpg
380017,False,537274,Предел возможного,0.62,False,https://www.themoviedb.org/movie/537274,https://www.themoviedb.org/t/p/w500//h9aXCUZ6g...,[18],h9aXCUZ6gj6V5NFv8a31J1OiDjq.jpg
380020,False,537277,母親,0.60,False,https://www.themoviedb.org/movie/537277,https://www.themoviedb.org/t/p/w500//3Ogn8SWj2...,[18],3Ogn8SWj29GpGHNSIkMYiKZdeBL.jpg
380022,False,537281,1+1=3 Ora Tinjon,0.60,False,https://www.themoviedb.org/movie/537281,https://www.themoviedb.org/t/p/w500//47yYJPL2q...,[18],47yYJPL2qU9LNQ2rTMs7p3RtfR7.jpg


 Create ImageGenerators

In [12]:
datagen = ImageDataGenerator(rescale=1 / 255., validation_split=0.1)

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=DATA_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",    
    target_size=(299, 299),
    subset='training',
    validate_filenames=True
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=DATA_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(299, 299),
    subset='validation',
    validate_filenames=True
)

Found 192488 validated image filenames belonging to 19 classes.
Found 21387 validated image filenames belonging to 19 classes.


In [13]:
# show class indicies
print(train_generator.class_indices)
print('length:', len(train_generator.class_indices))


{12: 0, 14: 1, 16: 2, 18: 3, 27: 4, 28: 5, 35: 6, 36: 7, 37: 8, 53: 9, 80: 10, 99: 11, 878: 12, 9648: 13, 10402: 14, 10749: 15, 10751: 16, 10752: 17, 10770: 18}
length: 19


In [14]:
#https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras
from sklearn.utils import class_weight
#In order to calculate the class weight do the following

class_weights = class_weight.compute_class_weight('balanced',list(train_generator.class_indices.keys()), df.genre_id.explode())
                                                 #np.unique(y_train),
                                                 #y_train)
class_weights_genre_id = dict(zip(list(train_generator.class_indices), class_weights))
display(class_weights_genre_id)
class_weights = dict(zip(list(range(len(class_weights))), class_weights))
class_weights

{12: 1.8201379050551538,
 14: 2.3873523093447906,
 16: 1.2595452291563363,
 18: 0.23875715761221894,
 27: 1.0941426662104918,
 28: 0.976952256204232,
 35: 0.3624541454023912,
 36: 3.3418158308211607,
 37: 5.32697008915732,
 53: 0.9921990089728138,
 80: 1.4069428487328732,
 99: 0.464520760663522,
 878: 2.2661809987968757,
 9648: 2.589777798492246,
 10402: 1.1858711861512974,
 10749: 0.9050347232826176,
 10751: 1.6533144393738215,
 10752: 4.401891370005447,
 10770: 2.5843424976890477}

{0: 1.8201379050551538,
 1: 2.3873523093447906,
 2: 1.2595452291563363,
 3: 0.23875715761221894,
 4: 1.0941426662104918,
 5: 0.976952256204232,
 6: 0.3624541454023912,
 7: 3.3418158308211607,
 8: 5.32697008915732,
 9: 0.9921990089728138,
 10: 1.4069428487328732,
 11: 0.464520760663522,
 12: 2.2661809987968757,
 13: 2.589777798492246,
 14: 1.1858711861512974,
 15: 0.9050347232826176,
 16: 1.6533144393738215,
 17: 4.401891370005447,
 18: 2.5843424976890477}

In [15]:
map_gender={"28":"Action",
"12":"Adventure",
"16":"Animation",
"35":"Comedy",
"80":"Crime",
"99":"Documentary",
"18":"Drama",
"10751":"Family",
"14":"Fantasy",
"36":	"History",
"27":"Horror",
"10402"	:"Music",
"9648":"Mystery",
"10749":"Romance",
"878"	:"Science Fiction",
"10770":"TV Movie",
"53":"Thriller",
"10752":"War",
"37":"Western"}

series_genre_id_counts = df.genre_id.explode().value_counts()
series_genre_id_counts
df_genre = pd.DataFrame(series_genre_id_counts)
df_genre["id"] = df_genre.index
df_genre.rename(columns={"genre_id" : "count"},inplace=True)
df_genre["name"] = df_genre["id"].apply(lambda x : map_gender[str(x)])
df_genre["weight"] = df_genre["id"].apply(lambda x : class_weights_genre_id[x])
df_genre.sort_values(by="count")

,count,id,name,weight
37,4392,37,Western,5.326970
10752,5315,10752,War,4.401891
36,7001,36,History,3.341816
9648,9034,9648,Mystery,2.589778
10770,9053,10770,TV Movie,2.584342
14,9800,14,Fantasy,2.387352
878,10324,878,Science Fiction,2.266181
12,12854,12,Adventure,1.820138
10751,14151,10751,Family,1.653314
80,16629,80,Crime,1.406943


# Simple Model

In [16]:
model = keras.Sequential(
    [ 
        layers.Conv2D(32, (3, 3), padding='same', input_shape=(299, 299, 3)),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(train_generator.class_indices), activation='sigmoid')
        #layers.Dense(len(train_generator.class_indices), activation='softmax')

    ])

In [17]:
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])

# Train

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    steps_per_epoch=1430,
    class_weight = class_weights
)

Epoch 1/10
  16/1430 [..............................] - ETA: 3:26:07 - loss: 0.4653 - accuracy: 0.0938

In [ ]:
#num_samples // batch_size
183069 / 32 / 4